4.1.2 파이토치로 MLP 구현하기

In [1]:
# 파이토치를 사용한 MLP
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
    """
    """
    def __init__(self, input_size, hidden_size=2, output_size=3, 
                 num_hidden_layers=1, hidden_activation=nn.Sigmoid):
        """가중치 초기화

        매개변수:
            input_size (int): 입력 크기
            hidden_size (int): 은닉층 크기
            output_size (int): 출력 크기
            num_hidden_layers (int): 은닉층 개수
            hidden_activation (torch.nn.*): 활성화 함수
        """
        super(MultilayerPerceptron, self).__init__()
        self.module_list = nn.ModuleList()
        
        interim_input_size = input_size
        interim_output_size = hidden_size
        
        for _ in range(num_hidden_layers):
            self.module_list.append(nn.Linear(interim_input_size, interim_output_size))
            self.module_list.append(hidden_activation())
            interim_input_size = interim_output_size
            
        self.fc_final = nn.Linear(interim_input_size, output_size)
        
        self.last_forward_cache = []
       
    def forward(self, x, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 함수를 위한 플래그
                크로스 엔트로피 손실을 사용하려면 반드시 False로 지정해야 합니다
        반환값:
            결과 텐서. tensor.shape는 (batch, output_dim)입니다.
        """
        self.last_forward_cache = []
        self.last_forward_cache.append(x.to("cpu").numpy())

        for module in self.module_list:
            x = module(x)
            self.last_forward_cache.append(x.to("cpu").data.numpy())
            
        output = self.fc_final(x)
        self.last_forward_cache.append(output.to("cpu").data.numpy())

        if apply_softmax:
            output = F.softmax(output, dim=1)
            
        return output

In [2]:
# MLP 객체 생성
batch_size = 2 # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)

print(mlp)

MultilayerPerceptron(
  (module_list): ModuleList(
    (0): Linear(in_features=3, out_features=100, bias=True)
    (1): Sigmoid()
  )
  (fc_final): Linear(in_features=100, out_features=4, bias=True)
)


In [3]:
# 랜덤한 입력으로 MLP 테스트하기
import torch

def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

x_input = torch.rand(batch_size, input_dim)
describe(x_input)

print()

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.4944, 0.5987, 0.6043],
        [0.3881, 0.9355, 0.4304]])

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[ 0.5920, -0.1105, -0.1509,  0.0463],
        [ 0.6112, -0.1185, -0.1382,  0.0492]], grad_fn=<AddmmBackward>)


In [4]:
# MLP 분류기로 확률 출력하기(apply_softmax=True)
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[0.3921, 0.1942, 0.1865, 0.2272],
        [0.3961, 0.1909, 0.1872, 0.2258]], grad_fn=<SoftmaxBackward>)


4.2.1 성씨 데이터셋

In [5]:
# SurnameDataset.__getitem__() 구현
class SurnameDataset(Dataset):
    # [코드 3-14]와 구현이 매우 비슷합니다.

    def __getitem__(self, index):           
            row = self._target_df.iloc[index]

            surname_vector = self._vectorizer.vectorize(row.surname)

            nationality_index = self._vectorizer.nationality_vocab.lookup_token(row.nationality)

            return {'x_surname': surname_vector, 'y_nationality': nationality_index}

In [ ]:
# SurnameVectorizer 구현
class SurnameVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """
    def __init__(self, surname_vocab, nationality_vocab):
        """
        매개변수:
            surname_vocab (Vocabulary): 문자를 정수에 매핑하는 Vocabulary 객체
            nationality_vocab (Vocabulary): 국적을 정수에 매핑하는 Vocabulary 객체
        """
        self.surname_vocab = surname_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname):
        """ 성씨에 대한 원-핫 벡터를 만듭니다

        매개변수:
            surname (str): 성씨
        반환값:
            one_hot (np.ndarray): 원-핫 벡터
        """
        vocab = self.surname_vocab
        one_hot = np.zeros(len(vocab), dtype=np.float32)
        
        for token in surname:
            one_hot[vocab.lookup_token(token)] = 1

        return one_hot

    @classmethod
    def from_dataframe(cls, surname_df):
        """ 데이터셋 데이터프레임에서 Vectorizer 객체를 만듭니다
        
        매개변수:
            surname_df (pandas.DataFrame): 성씨 데이터셋
        반환값:
            SurnameVectorizer 객체
        """
        surname_vocab = Vocabulary(unk_token="@")
        nationality_vocab = Vocabulary(add_unk=False)

        for index, row in surname_df.iterrows():
            for letter in row.surname:
                surname_vocab.add_token(letter)
                
            nationality_vocab.add_token(row.nationality)

        return cls(surname_vocab, nationality_vocab)

In [ ]:
# MLP 기반의 SurnameClassifier
class SurnameClassifier(nn.Module):
    """ 성씨 분류를 위한 다층 퍼셉트론 """
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(SurnameClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate_vector = F.relu(self.fc1(x_in))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

4.2.4 모델 훈련

In [ ]:
# MLP 기반의 성씨 분류기를 위한 하이퍼파라미터와 프로그램 설정
args = Namespace(
    # 날짜와 경로 정보
    surname_csv="data/surnames/surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch4/surname_mlp",
    # 모델 하이퍼파라미터
    hidden_dim=300,
    # 훈련 하이퍼파라미터
    seed=1337,
    num_epochs=100,
    early_stopping_criteria=5,
    learning_rate=0.001,
    batch_size=64,
    # 실행 옵션
    cuda=False,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

In [ ]:
# 데이터셋 모델, 손실, 옵티마이저 생성
dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
vectorizer = dataset.get_vectorizer()

classifier = SurnameClassifier(input_dim=len(vectorizer.surname_vocab), 
                               hidden_dim=args.hidden_dim, 
                               output_dim=len(vectorizer.nationality_vocab))
classifier = classifier.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

In [ ]:
# 훈련 반복 코드의 일부
# 훈련 과정은 5단계입니다

# --------------------------------------
# 단계 1. 그레이디언트를 0으로 초기화합니다
optimizer.zero_grad()

# 단계 2. 출력을 계산합니다
y_pred = classifier(batch_dict['x_surname'])

# 단계 3. 손실을 계산합니다
loss = loss_func(y_pred, batch_dict['y_nationality'])
loss_t = loss.item()
running_loss += (loss_t - running_loss) / (batch_index + 1)

# 단계 4. 손실을 사용해 그레이디언트를 계산합니다
loss.backward()

# 단계 5. 옵티마이저로 가중치를 업데이트합니다
optimizer.step()

4.2.5 모델 평가와 예측

In [ ]:
# 기존 모델(분류기)을 사용한 추론: 주어진 이름의 국적 예측하기
def predict_nationality(surname, classifier, vectorizer):
    vectorized_surname = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).view(1, -1)
    result = classifier(vectorized_surname, apply_softmax=True)

    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'nationality': predicted_nationality, 'probability': probability_value}

In [ ]:
# 최상위 국적 k개 예측하기
def predict_topk_nationality(name, classifier, vectorizer, k=5):
    vectorized_name = vectorizer.vectorize(name)
    vectorized_name = torch.tensor(vectorized_name).view(1, -1)
    prediction_vector = classifier(vectorized_name, apply_softmax=True)
    probability_values, indices = torch.topk(prediction_vector, k=k)
    
    # 반환되는 크기는 (1,k)입니다
    probability_values = probability_values.detach().numpy()[0]
    indices = indices.detach().numpy()[0]
    
    results = []
    
    for prob_value, index in zip(probability_values, indices):
        nationality = vectorizer.nationality_vocab.lookup_index(index)
        results.append({'nationality': nationality, 
                        'probability': prob_value})
    
    return results

4.2.6 MLP 규제: 가중치 규제와 구조적 규제(또는 드롭아웃)

In [ ]:
# 드롭아웃을 적용한 MLP
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

batch_size = 2 # 동시에 입력될 샘플의 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)